In [15]:
#data load and preprocessing
import numpy as np
import pandas as pd 
import re 
import string

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import nltk
from nltk.probability import FreqDist
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#ML Prepration
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score 

# Classification Model
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LEXUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEXUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEXUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
# reading the file
df = pd.read_csv("train.csv")
df.head()

,Tweet,Class
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,normal
1,سورية بلد الحضارات تربطها بعلية او بحيوان,normal
2,اخي الحاج اذا شعرت انك محرجا من الانتقادات لتص...,normal
3,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,normal
4,هذا البطل الذي قاتل وجاذف بحياته لتحيا انت يا ...,abusive


In [17]:
df.Tweet[50]

'هذا أحد أهم عوامل الدمار الداخلي للبلداننا '

In [18]:
# define punctuations
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' 

# Expande Arabic StopWords
arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))
data = []
with open('arabic_stop_words.txt',encoding="utf8") as myfile:
    for line in myfile:
        data.extend(map(str, line.rstrip('\n').split(',')))
arabic_stopwords.update(data)

#difine Arabic text stemmer
st = ISRIStemmer()

# 
def clean_text(text):
    text = text.translate(str.maketrans('','', punctuations))
    text = ' '.join([word for word in text.split() if word not in arabic_stopwords])
    text = st.stem(text)
    return text

In [19]:
df['Tweet'] = df['Tweet'].apply(clean_text)

In [20]:
#Create 
max_features = 5000
count_vector = CountVectorizer(max_features = max_features)  
PoemBOW = count_vector.fit_transform(df['Tweet']).toarray() 
PoemBOW

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [21]:
# BOW frequency
print(count_vector.vocabulary_)

{'وزير': 4559, 'الخارجية': 580, 'اللبناني': 969, 'جبران': 2076, 'باسيل': 1271, 'تغريداته': 1960, 'القمة': 937, 'العربية': 837, 'التنموية': 481, 'الاقتصادية': 403, 'والاجتماعية': 4400, 'سورية': 2699, 'بلد': 1645, 'اخي': 195, 'الحاج': 524, 'شعرت': 2786, 'انك': 1207, 'محرجا': 3842, 'الانتقادات': 420, 'لتصريح': 3595, 'داعي': 2416, 'تهاجم': 2041, 'المنتقد': 1057, 'فيك': 3311, 'تعيش': 1957, 'بلا': 1638, 'فتن': 3228, 'ليل': 3746, 'نهار': 4254, 'وبكرة': 4489, 'قلهم': 3414, 'الموضوع': 1073, 'السيد': 721, 'بطل': 1556, 'قاتل': 3327, 'بحياته': 1463, 'واطي': 4386, 'عيب': 3184, 'الشوم': 752, 'ولعها': 4684, 'وهيك': 4735, 'بتكون': 1428, 'الشقيقة': 743, 'قدمت': 3359, 'اكبر': 301, 'خدمة': 2338, 'تترحم': 1824, 'وسام': 4563, 'الحسن': 546, 'الحريري': 543, 'والحريري': 4416, 'عارف': 3007, 'يأخذ': 4760, 'كلنا': 3524, 'معك': 3976, 'معالي': 3964, 'الوزير': 1139, 'الملك': 1051, 'ولك': 4685, 'مبارح': 3788, 'المريض': 1001, 'النفسي': 1099, 'كنت': 3539, 'عم': 3123, 'تدافع': 1868, 'العماد': 857, 'عثمان': 3045, 'لأنو'

In [22]:
# encode Label Column
encoder = LabelEncoder()
Label_encoder = encoder.fit_transform(df['Class'])
Label_encoder

array([2, 2, 2, ..., 2, 0, 2])

In [23]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(PoemBOW,Label_encoder, test_size =0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3740, 5000), (936, 5000), (3740,), (936,))

In [24]:
logModel = LogisticRegression()
# train model
logModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = logModel.predict(X_test) 

# print the accuracy and classification_report
print('Test model accuracy: ',accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test model accuracy:  0.7916666666666666
              precision    recall  f1-score   support

           0       0.72      0.58      0.64       264
           1       0.73      0.29      0.42        65
           2       0.82      0.94      0.87       607

    accuracy                           0.79       936
   macro avg       0.75      0.60      0.64       936
weighted avg       0.78      0.79      0.78       936



In [25]:
MNBModel = MultinomialNB()
# train model
MNBModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = MNBModel.predict(X_test) 

# print the accuracy and classification_report
print('Test model accuracy: ',accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test model accuracy:  0.7553418803418803
              precision    recall  f1-score   support

           0       0.62      0.70      0.66       264
           1       0.34      0.28      0.31        65
           2       0.86      0.83      0.85       607

    accuracy                           0.76       936
   macro avg       0.61      0.60      0.60       936
weighted avg       0.76      0.76      0.75       936



In [26]:
RandomModel = RandomForestClassifier()
# train model
RandomModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = RandomModel.predict(X_test) 

# print the accuracy and classification_report
print('Test model accuracy: ',accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test model accuracy:  0.7852564102564102
              precision    recall  f1-score   support

           0       0.68      0.61      0.65       264
           1       0.70      0.22      0.33        65
           2       0.82      0.92      0.87       607

    accuracy                           0.79       936
   macro avg       0.74      0.58      0.62       936
weighted avg       0.78      0.79      0.77       936



In [27]:
SVCModel = LinearSVC()

# train model
SVCModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = SVCModel.predict(X_test) 

# print the accuracy and classification_report
print('Test model accuracy: ',accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Test model accuracy:  0.7660256410256411
              precision    recall  f1-score   support

           0       0.63      0.64      0.64       264
           1       0.59      0.35      0.44        65
           2       0.83      0.87      0.85       607

    accuracy                           0.77       936
   macro avg       0.69      0.62      0.64       936
weighted avg       0.76      0.77      0.76       936



In [28]:
GNBModel = GaussianNB()

# train model
GNBModel.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = GNBModel.predict(X_test) 

# print the accuracy and classification_report
print('Test model accuracy: ',accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test model accuracy:  0.5801282051282052
              precision    recall  f1-score   support

           0       0.44      0.50      0.46       264
           1       0.11      0.25      0.15        65
           2       0.81      0.65      0.72       607

    accuracy                           0.58       936
   macro avg       0.45      0.46      0.45       936
weighted avg       0.66      0.58      0.61       936



In [34]:
df2 = pd.read_csv('train.csv')

#qoute
test_text = [df2['Tweet'][55]]
# حزن
# encoding
test_vector = count_vector.transform(test_text)
test_vector = test_vector.toarray()

## Perform and Evaluate the Model 
text_predict_class = encoder.inverse_transform(logModel.predict(test_vector))
print(test_text,'\n',text_predict_class)

['اودع مليار عشان حزب الشيطان بلبنان يمر بمرحله صعبة بعد ما توقف الدعم الايران اثر الحصار عليها'] 
 ['hate']


In [44]:
df.Tweet[100]

'جبران باسيل وعمه جنرال اسرائيل سابقا العماد ميشال عون يمثلون سياسة الزباله قدمها سيدهم نصراللات طبقا الخياسة'